In [25]:
SHOW_OUTPUTS = True

def ShowTableau(c, a, b, certificate, optimal):
    print("\n")
    print("c[]: {}".format(c))
    print("A[] : {}".format(a))
    print("b[] : {}".format(b))
    print("Optimal : {}".format(optimal))
    print("\n")

def Identity(n):
    m=[[0 for x in range(n)] for y in range(n)]
    for i in range(0,n):
        m[i][i] = 1
    return m

def Pivot(which_row, which_column, c, a, b, certificate, operations, optimal):
    
    pivot = a[which_row][which_column]
    if(SHOW_OUTPUTS): print("Pivot: {}".format(pivot))
    
    if(SHOW_OUTPUTS): print("Iterate A[{},{}] over the pivot: {}".format(which_row, which_column ,pivot))
    for i in range(len(a[which_row])):
        if(a[which_row][i] != 0):
            a[which_row][i] = a[which_row][i]/pivot
            
    if(SHOW_OUTPUTS): print("Iterate operations[{},{}] over the pivot: {}".format(which_row, which_column ,pivot))
    for i in range(len(operations[which_row])):
        if(operations[which_row][i] != 0):
            operations[which_row][i] = operations[which_row][i]/pivot
    
    if(SHOW_OUTPUTS): print("Iterate b[{}] over the pivot: {}".format(which_row, pivot))
    if(b[which_row] != 0):
        b[which_row][0] = b[which_row][0]/pivot
        
    if(SHOW_OUTPUTS): print("Stating pivotation")
        
    if(SHOW_OUTPUTS): print("Pivoting A[]")
    for i in range(len(a)):
        for j in range(len(a[which_row])):
            if(SHOW_OUTPUTS): print("i = {}, j = {}".format(i,j))
            if(i != which_row):
                if(a[i][j] != 0 and a[which_row][j] != 0):
                    value = (a[i][j]/a[which_row][j])
                    if(SHOW_OUTPUTS): print("Iterating the value:{} over A[{},{}]".format(a[which_row][j], i, j))
                    a[i][j] += (value * -1) 
                    if(SHOW_OUTPUTS): print(a)
                    
                    if(b[i][0] != 0):
                        value = (b[i][0]/b[which_row][0])
                        if(SHOW_OUTPUTS): print("Iterating the value:{} over b[{}]".format(a[which_row][j], i))
                        b[i][0] += (value * -1)
                        if(SHOW_OUTPUTS): print(b)
                        
            if(c[j] != 0 and a[which_row][j] != 0):
                value = (c[j]/pivot) * -1
                if(SHOW_OUTPUTS): print("Iterating the value:{} over c[{}]".format(a[which_row][j], j))
                c[j] += value
                if(SHOW_OUTPUTS): print(c)
                
        
    return(c, a, b, certificate, operations, optimal)
    

def SimplexTableau(r, v, c, a, b):
    certificate = [0] * r
    operations = Identity(3)
    gap_vars = Identity(3)
    
    for i in range(len(a)):
        a[i] = a[i] + gap_vars[i] 

    c = c + [0] * r
    optimal = 0
    
    for i in range(len(c)):
        c[i] = c[i] * -1
        
    for c_index in range(len(c)):
        
        if(SHOW_OUTPUTS): ShowTableau(c, a, b, certificate, optimal)
        
        if(c[c_index] < 0 ):
            lower = 10000
            for i in range(len(b)):
                if(b[i][0] != 0):
                    if(SHOW_OUTPUTS): print("Minimizing: {}/{}".format(b[i][0],a[i][c_index]))
                    if(a[i][c_index] != 0 and b[i][0]/a[i][c_index] < lower):
                        lower = b[i][0]/a[i][c_index]
                        which_row = i
                        which_column = c_index
            if(SHOW_OUTPUTS): print("Lower value is {} on row #{}".format(lower, which_row))
            c, a, b, certificate, operations, optimal = Pivot(which_row, which_column, c, a, b, certificate, operations, optimal)

        

In [26]:
r = 3
v = 3
c = [2,4,8]
a = [[1,0,0],[0,1,0],[0,0,1]]
b = [[1],[1],[1]]
SimplexTableau(r,v,c,a,b)



c[]: [-2, -4, -8, 0, 0, 0]
A[] : [[1, 0, 0, 1, 0, 0], [0, 1, 0, 0, 1, 0], [0, 0, 1, 0, 0, 1]]
b[] : [[1], [1], [1]]
Optimal : 0


Minimizing: 1/1
Minimizing: 1/0
Minimizing: 1/0
Lower value is 1.0 on row #0
Pivot: 1
Iterate A[0,0] over the pivot: 1
Iterate operations[0,0] over the pivot: 1
Iterate b[0] over the pivot: 1
Stating pivotation
Pivoting A[]
i = 0, j = 0
Iterating the value:1.0 over c[0]
[0.0, -4, -8, 0, 0, 0]
i = 0, j = 1
i = 0, j = 2
i = 0, j = 3
i = 0, j = 4
i = 0, j = 5
i = 1, j = 0
i = 1, j = 1
i = 1, j = 2
i = 1, j = 3
i = 1, j = 4
i = 1, j = 5
i = 2, j = 0
i = 2, j = 1
i = 2, j = 2
i = 2, j = 3
i = 2, j = 4
i = 2, j = 5


c[]: [0.0, -4, -8, 0, 0, 0]
A[] : [[1.0, 0, 0, 1.0, 0, 0], [0, 1, 0, 0, 1, 0], [0, 0, 1, 0, 0, 1]]
b[] : [[1.0], [1], [1]]
Optimal : 0


Minimizing: 1.0/0
Minimizing: 1/1
Minimizing: 1/0
Lower value is 1.0 on row #1
Pivot: 1
Iterate A[1,1] over the pivot: 1
Iterate operations[1,1] over the pivot: 1
Iterate b[1] over the pivot: 1
Stating pivotation
P

In [98]:
print(a[0][3])

1.0
